In [2]:
import requests
import time
from bs4 import BeautifulSoup
import csv
from collections import defaultdict
import json
import sqlite3
from sqlite3 import Error

In [2]:
# terms_dict = defaultdict(list)
terms_list = []
appDictionary = {}
relatedSearchTerms = {}

In [77]:
#Class to store app details in object format
class AppDetails:
    def __init__(self, title, description, stars, appID, imageSource):
        self.title = title
        self.description = description
        self.stars = stars
        self.appID = appID
        self.imageSource = imageSource
        
    def toString(application):
        return application.title + ", " + application.appID + ", " + application.description + ", " + application.stars + ", " + application.imageSource

In [3]:
with open('android_terms.csv','rt')as f:
  data = csv.reader(f)
  lineNumber = 0;
  for row in data:
        if lineNumber >= 1:
            key = row[2];
            terms = row[3];
            i = 0;
            for term in terms.split('"'):
                if i%2 == 0:
                    i = i+1;
                    continue
                else:
                    i = i+1;
                    terms_list.append(term)
        else:
            lineNumber = lineNumber + 1;

In [4]:
finalTermsList = []

In [5]:
finalTermsList.clear()
for term in terms_list:
    result = "";
    if len(term.split(' ')) > 1:
        count = len(term.split(' '));
        i = 0;
        for word in term.split(' '):
            if i > 0:
                result = result + "+";
            result = result + word;
            i = i + 1;
        finalTermsList.append(result)
    else:
        finalTermsList.append(term)

In [6]:
len(finalTermsList)

432331

In [ ]:
finalTermsList

In [7]:
payload = {'q': 'mspy', 't': 'app'}
r = requests.get('https://apk.support/search', params=payload);
print(r.url)
print(r.status_code)

https://apk.support/search?q=mspy&t=app


In [9]:
# TEST
soup = BeautifulSoup(r.text, 'html.parser')
app_table = soup.find_all("h3")
appNamesList = []
for app in app_table:
    appName = app.get_text()
    appNamesList.append(appName)
appDictionary["mspy"] = appNamesList

In [14]:
# TEST Cleanup List
for term in finalTermsList:
    if(term == ""):
        finalTermsList.remove(term)
singleWordSet = set();
for term in finalTermsList:
    if(term.find("+") == -1 and term != ""):
        singleWordSet.add(term)

KeyboardInterrupt: 

In [7]:
wordSet = set();
for term in finalTermsList:
    if(term != ""):
        wordSet.add(term)
print(len(wordSet))

1557


In [70]:
# FOR TEST PURPOSE
singleWordSet = {'cheater', 'cheaters', 'cheating', 'track husband', 'keylogger', 'keyloggers'}

In [ ]:
i = 0
for word in wordSet:
    time.sleep(1)
    print("Starting " + word + " " + str(i))
    payload = {'q': word, 't': 'app'};
    r = requests.get('https://apk.support/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "it_column"})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    for name in names_table:
        information = name.find_all("a")
        titleTag = information[0].find_all("h3")
        title = titleTag[0].get_text()
#         print("Title is " + title)
        descriptionTag = information[0].find_all("p")
        description = descriptionTag[0].get_text()
#         print("Description is " + description)
        starsTag = information[0].find_all("div", attrs = {"class" : "stars"})
        starsSpan = starsTag[0].find_all("span")
        stars = starsSpan[0]['title']
        starCount = stars[stars.rindex(' ')+1:]
#         print("Stars is " + starCount)
        
        appID = information[0]['href']
        appID = appID[4 : ]
#         print("AppID:- " + appID)
        imageTag = information[0].find_all("div", attrs={"class" : "seo_img"})
        imageTag = imageTag[0].find_all("img")
        imageSource = imageTag[0]['data-original']
#         print(" ImageSrc:- " + imageSource)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource)
        finalList.append(perAppObject)
        
    # Suggestion Addition
    suggestionList = soup.find_all("div", attrs={"class": "suggest"})
    suggestionList = suggestionList[0].find_all("li")
    suggestions = []
    for suggestion in suggestionList:
        suggestionName = suggestion.get_text()
        suggestions.append(suggestionName)
    appDictionary[word] = finalList
    relatedSearchTerms[word] = suggestions
    i = i+1
    print("Completed " + word)

In [ ]:
for app in appDictionary:
    appList = appDictionary[app]
    print("Entries for " + app)
    for application in appList:
        print(application.toString())
        print("------------------------------->>>")

In [ ]:
# Saving dict to file
with open('apksupport.json', 'w') as f:
    json.dump(appDictionary, f)

with open('relatedSearchTerms_apksupport.json', 'w') as f:
    json.dump(relatedSearchTerms, f)

In [ ]:
SQLite3
dataset package python

upper table -> key and set of app ids
lower table -> app id is primary key and we write to this.

In [18]:
#DB DRIVERS

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn
            
            
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
        
sql_create_appDetailsMainTable_table = """ CREATE TABLE IF NOT EXISTS appDetailsMainTable (
                                        key text PRIMARY KEY,
                                        appIDs text NOT NULL,
                                        source text NOT NULL
                                    ); """

sql_create_appDetails_table = """ CREATE TABLE IF NOT EXISTS appDetails (
                                        appID text PRIMARY KEY,
                                        title text NOT NULL,
                                        description text,
                                        stars text,
                                        imageSource text,
                                        source text
                                    ); """

sql_create_relatedSearchTerms_table = """ CREATE TABLE IF NOT EXISTS relatedSearchTerms (
                                        keyTerm text PRIMARY KEY,
                                        terms text NOT NULL,
                                        source text
                                    ); """

In [20]:
# Create connection
conn = create_connection(r"database.db")

# Create table
if conn is not None:
    create_table(conn, sql_create_appDetailsMainTable_table)
    create_table(conn, sql_create_appDetails_table)
    create_table(conn, sql_create_relatedSearchTerms_table)
else:
    print("Error! cannot create the database connection.")